In [3]:
import pandas as pd

In [ ]:
dtype={"user_id": int, "username": object}

In [13]:
inspections = pd.read_csv('data/potential_inspection_summary_data.csv', sep=';')
violations = pd.read_csv('data/potential_violation_details_data.csv', sep=';')
locations = pd.read_csv('data/inspections_geocoded.csv', sep=';')

In [36]:
len(inspections.drop_duplicates())

68362

In [44]:
print len(inspections)
print len(inspections.drop_duplicates(inspections.columns.difference(['inspection_id'])))
print len(inspections[inspections['known_valid'] == True])
print len(inspections[inspections['known_valid'] == True].drop_duplicates(inspections.columns.difference(['inspection_id'])))


68363
62318
36988
36960


In [41]:
inspections[inspections.duplicated(inspections.columns.difference(['inspection_id']))].head(20).sort_values('inspection_date')

,inspection_id,establishment_name,address,telephone,email,inspection_date,inspection_time_in,inspection_time_out,license_holder,license_number,...,critical_violations,critical_violations_corrected_on_site,critical_violations_repeated,noncritical_violations,noncritical_violations_corrected_on_site,noncritical_violations_repeated,inspector_comments,inspector_name,inspector_badge_number,known_valid
521,522,LANGSTON BAR AND GRILLE (n) (d),"1831 BENNING RD NE WASHINGTON, DC 20002",(202) 397-3637,NaN,01/01/2010,NaN,NaN,Langston Bar & Grille LLC,93130xxxx-68002762,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,BRB,NaN,False
184,185,THE RED PALACE (n),"1212 H ST NE Washington, DC 20002",(202) 399-3201,NaN,01/07/2008,NaN,NaN,Red & Black LLC,20260xxxx-66003884,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AG,NaN,False
309,310,RAINBOW CITY CARRYOUT,"1320 H ST NE WASHINGTON, DC 20002",NaN,NaN,02/07/2008,NaN,NaN,"Ru Chen, Xing",21461xxxx-53002146,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,CB,NaN,False
470,471,MONTANA LIQUOR,"1801 MONTANA AVE NE Washington, DC 20002",(202) 635-0975,faraw28@gmail.com,02/11/2009,NaN,NaN,Zeb & Seble LLC,20050xxxx-64004236,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,BRB,NaN,False
471,472,MONTANA LIQUOR,"1801 MONTANA AVE NE Washington, DC 20002",(202) 635-0975,faraw28@gmail.com,02/11/2009,NaN,NaN,Zeb & Seble LLC,20050xxxx-64004236,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,BRB,NaN,False
235,236,GOOD DANNY'S CARRYOUT,"610 H ST NE Washington, DC 20002",NaN,NaN,03/11/2008,NaN,NaN,Danny & Company Inc,00000xxxx-39500368,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AG,NaN,False
438,439,SAFEWAY #4270,"1601 MARYLAND AVE NE WASHINGTON, DC 20002",(202) 398-6903,S4270c90@safeway.com,05/06/2008,NaN,NaN,Safeway Inc,09305xxxx-15000092,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AG,NaN,False
138,139,CAPITOL CASH & CARRY,"1110 OKIE ST NE Washington, DC 20002",(202) 249-1110,Scott.capitolcashandcarry@gmail.com,05/11/2008,NaN,NaN,D C Cash & Carry LLC,09308xxxx-39506144,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,BF,NaN,False
43,44,Trinity Washington University Center Pool,"125 MICHIGAN AVE NE WASHINGTON, DC 20017",(202) 884-9091,NaN,07/10/2008,NaN,NaN,Trinity College,09211xxxx-68002107,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
563,564,KENTUCKY FRIED CHICKEN STORE #Y075018,"1944 BLADENSBURG RD NE Washington, DC 20002",(202) 526-8563,k071133@kfc.com,08/03/2010,NaN,NaN,KFC US Properties Inc,09313xxxx-13000022,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,JA,NaN,False


In [17]:
len(violations)

202232

In [18]:
len(violations.drop_duplicates())

202195

In [23]:
violations[violations.duplicated()]

,inspection_id,violation_number,violation_text,dcmr_25_code
1036,7366,8,Separate hot and cold water faucets at handwas...,2411.2
3678,8483,53,The floors in the restroom and storage area ne...,3202.2
12421,21337,33,Improper thawing. (Corrected On Site),1002.1
19108,98189,1,Certified Food Protection Manager certificate ...,NaN
19110,98189,8,No handwashing signage at the handwashing sink...,3004.1
39093,130113,14,THE ICE MACHINE CONTAINED CALCIUM BUILD-UP BEH...,1900.1
39857,136110,14,"Unclean food-contact surfaces -Cutting boards,...",1900.1
43360,136702,8,No handwashing signage at handwashing sink. (C...,3004.1
55105,168389,14,Unclean food-contact surfaces (Corrected On Site),1900.1
56521,168961,62,Inspection report not posted.(CORRECT VIOLATIO...,4310.2


In [20]:
violations.head(30)

,inspection_id,violation_number,violation_text,dcmr_25_code
0,292,45,"Dirty cabinets, in storage area.",1700.1
1,292,53,"DIRTY FLOORS,LITERS IN SERVICE AREA",2900.1
2,485,21,FOOD IN WALK IN NOT LABELED AS TO DATE OF PREP...,1007.1
3,485,34,THERMOMETERS NOT IN ALL COOLING UNITS PROVIDE ...,1510.1
4,485,49,NON FUNCTIONING HAND SINK IN KITCHEN(CORRECT V...,2401.1
5,485,51,TOILET DOOR AJAR.MUST PROVIDE A SELF CLOSING D...,3208.1
6,485,54,LOW LIGHT INTENSITY OVER STOVE. REPLACE ALL BU...,3008.1(a)
7,685,13,"Uncovered chicken breasts, turkey , and roasts...",802.1(a)-(b)
8,685,14,Utensils run through dish washer are not being...,2001.1
9,685,18,Meatloaf and roast beef that were cooked last ...,1003.1


In [19]:
violations['violation_number'].value_counts()

53    11961
47     9452
8      9275
14     9083
36     7843
2      7743
49     7397
38     6528
20     6357
45     6229
51     6051
46     5943
1      5832
21     5522
10     5442
37     5386
34     5219
13     5162
39     4977
55     4790
48     4745
5      4035
16     4024
54     3795
35     3696
3      3441
22     3392
23     2966
41     2944
50     2890
      ...  
56     2118
6      2104
31     1821
33     1815
19     1694
40     1152
42     1118
26     1059
44      793
7       669
28      641
12      614
4       592
27      514
29      477
32      401
9       386
30      382
18      306
24      306
11      282
25      180
17      114
59       60
66       34
63       29
58       15
61        4
60        1
67        1
Name: violation_number, Length: 64, dtype: int64

In [21]:
violations[violations['violation_number'] == 53].head(20)

,inspection_id,violation_number,violation_text,dcmr_25_code
1,292,53,"DIRTY FLOORS,LITERS IN SERVICE AREA",2900.1
27,1044,53,NO DRESSING AREA FOR EMPLOYEES. DESIGNATE AN A...,3010.1
32,1819,53,Excess ice build-up in the floor of the walk-i...,3201.1
50,3045,53,Observed old or unused equipment stored throug...,3201.1
54,4025,53,Clean floor along the wall and replace the mis...,3201.1
71,4660,53,Drink spills and other food debris on floor an...,3201.1
160,7110,53,"Observed, the following items in need of cleaning",NaN
161,7110,53,"Observed, mops stored improperly (CORRECT VIOL...",3205.1
175,7111,53,Grease accumulation on walls next to ventilati...,3201.1
207,7119,53,Leaking ceiling near open display freezer (mea...,3200.1


In [24]:
inspections.groupby('license_number').head(20)

,inspection_id,establishment_name,address,telephone,email,inspection_date,inspection_time_in,inspection_time_out,license_holder,license_number,...,critical_violations,critical_violations_corrected_on_site,critical_violations_repeated,noncritical_violations,noncritical_violations_corrected_on_site,noncritical_violations_repeated,inspector_comments,inspector_name,inspector_badge_number,known_valid
0,1,Tigani Hair Boutique,"2130 P ST NW WASHINGTON, DC 20037",(202) 466-2626,NaN,29/01/2009,NaN,NaN,Tigani P Street LLC,09202xxxx-67003480,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
1,2,Chevy Chase Arcade Barber Shop,"5520 CONNECTICUT AVE NW WASHINGTON, DC 20015",NaN,NaN,26/09/2008,NaN,NaN,Chevy Chase Arcade Barber Shop,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NF,NaN,False
2,3,Chevy Chase Arcade Barber Shop,"5520 CONNECTICUT AVE NW WASHINGTON, DC 20015",NaN,NaN,20/11/2008,NaN,NaN,Chevy Chase Arcade Barber Shop,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NF,NaN,False
3,4,Chevy Chase House,"1845 M ST NE WASHINGTON, DC 20002",(202) 686-0045,NaN,18/02/2009,NaN,NaN,Chevy Chase House,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AG,NaN,False
4,5,SALINA RESTAURANT,"1936 9TH ST NW WASHINGTON, DC 20001",(202) 299-0511,terfneh.kahsay@gmail.com,18/09/2009,NaN,NaN,Salina Corporation,09313xxxx-12000109,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,CB,NaN,False
5,6,SALINA RESTAURANT,"1936 9TH ST NW WASHINGTON, DC 20001",(202) 299-0511,terfneh.kahsay@gmail.com,22/09/2009,NaN,NaN,Salina Corporation,09313xxxx-12000109,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,CB,NaN,False
6,7,LYON BAKERY,"135 R ST SW WASHINGTON, DC 20024",NaN,NaN,17/07/2009,NaN,NaN,Lyon Bakery Inc,09301xxxx-12000010,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
7,8,Manuel Salon,"3256 JONES CT NW WASHINGTON, DC 20007",(202) 340-8231,NaN,08/01/2009,NaN,NaN,Manuel Salon,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
8,9,Diane's Hair Stylist,"2645 NAYLOR RD SE Washington, DC 20019",NaN,NaN,28/08/2009,NaN,NaN,Diane's Hair Stylist,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,False
9,10,CLEVELAND PARK LIQUORS,"3423 CONNECTICUT AVE NW Washington, DC 20008",(202) 363-4265,mail2purewal@yahoo.com,09/12/2008,NaN,NaN,Tri-Dev Inc.,09306xxxx-39308925,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NF,NaN,False


In [25]:
len(inspections.groupby('license_number'))

7284

In [30]:
grouped = inspections.groupby('license_number')

In [33]:
for name, group in grouped:
    if len(group['license_holder'].unique()) > 1:
        print name
        for i in group['license_holder'].unique():
            print i
        print ''
        

01187xxxx-00000359
11641 - Abdelkarim Mazouz
ABDELKARIM MAZOUZ - 10977

04003xxxx-15001961
First Arman,Inc.
First Arman Inc

05107XXXX-50006170
2101 Wisc Associates
2101 WISC ASSOCIATES

09301xxxx-12000010
Lyon Bakery Inc
Lyon Bakery Inc.

09305XXXX-150000031
Quality Brand Capital, LLC
DDC 14-1101 LLC

09305xxxx-12000019
Pret Union Station Inc
Bean and Bite LLC

09305xxxx-12000048
Thaiger Crossing Inc
Thaiger Crossing Incorporated

09305xxxx-12000074
Jerome and Brenda and Associates, Inc
Jerome and Brenda And Associates INC

09305xxxx-13000015
Quick Shop
DDS/RSA Randolph-Sheppard Vending Facility Program

09305xxxx-14000033
Nova Frozen Yogurt, Llc
Gonzales Group Inc.

09305xxxx-15000002
Harper LLC
Harris Teeter, LLC

09305xxxx-15000062
Greenway Partnership LLC
RFK Inc

09305xxxx-15000068
Starbucks Corp
Starbucks Corporation

09305xxxx-15000140
Mary P Brockington,LLC
Mary Brockington LLC

09305xxxx-16000005
Silver Cinema Acquisition Co
Panera LLC

09305xxxx-16000025
Ivy City Smokehouse 

In [34]:
inspections['establishment_type'].value_counts()

Restaurant Total             37438
Delicatessen                 14966
School Cafeteria              3577
Food Products                 3272
Mobile Vending                2783
Grocery Store                 1978
Caterer                        888
Indoor Pool                    706
Hotel                          671
Beauty Shop                    429
Unlicensed Food                419
Bakery                         365
Beauty Shop-Nails              357
Barber Shop                    207
Marine-Food Prod (Retail)      120
Ice Cream Manufacturer          41
Marine-Food (Wholesale)         40
Beauty Shop-Braiding            36
Massage & Spa Treatment         26
Beauty Shop-Esthetics           25
Commission Merchant              9
Unknown                          7
Health Spa Sales                 2
Health Spa                       1
Name: establishment_type, dtype: int64

In [35]:
inspections['inspection_type'].value_counts()

Routine                       29888
Follow-up                     16158
Complaint                     11364
Preoperational                 3282
License Renewal                2219
Restoration                     903
Other                           795
Initial                         404
Ownership change                270
Follow up                       187
Summer Feeding                  169
Ownership Change                159
RESTORATION                     136
Change of Ownership             124
summer feeding                  124
Ownership  change               117
New ownership                    97
Sweep                            96
follow up                        88
HACCP                            85
SWEEP                            78
License Restoration              73
Fire                             51
CHANGE OF OWNERSHIP              47
ownership change                 46
OWNERSHIP CHANGE                 45
Renewal                          44
License Transfer            